In [12]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [13]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [14]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [15]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [16]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [17]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [18]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [19]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '957116', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '248591', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '247333', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [20]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [21]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [22]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [23]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [24]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
38627,CN0100100000000,1,1,"Autauga County, AL",Sep-20,"25,403","24,049","1,354",5.3,01001
38628,CN0100300000000,1,3,"Baldwin County, AL",Sep-20,"97,719","92,203","5,516",5.6,01003
38629,CN0100500000000,1,5,"Barbour County, AL",Sep-20,"9,595","8,750",845,8.8,01005
38630,CN0100700000000,1,7,"Bibb County, AL",Sep-20,"8,655","8,073",582,6.7,01007
38631,CN0100900000000,1,9,"Blount County, AL",Sep-20,"24,703","23,688","1,015",4.1,01009


In [25]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [26]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41841,CN7214500000000,72,145,"Vega Baja Municipio, PR",Sep-20,"12,794","11,165",1629,12.7,72145
41842,CN7214700000000,72,147,"Vieques Municipio, PR",Sep-20,"2,503","2,211",292,11.7,72147
41843,CN7214900000000,72,149,"Villalba Municipio, PR",Sep-20,"6,721","6,064",657,9.8,72149
41844,CN7215100000000,72,151,"Yabucoa Municipio, PR",Sep-20,"8,043","7,198",845,10.5,72151
41845,CN7215300000000,72,153,"Yauco Municipio, PR",Sep-20,"9,251","8,199",1052,11.4,72153


In [27]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [28]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [29]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [30]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [31]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [32]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [33]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [34]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [35]:
covid.tail(50)

,date,county,state,fips,cases,deaths
816613,2020-12-10,Ozaukee,Wisconsin,55089.0,6038,48.0
816614,2020-12-10,Pepin,Wisconsin,55091.0,552,5.0
816615,2020-12-10,Pierce,Wisconsin,55093.0,3046,25.0
816616,2020-12-10,Polk,Wisconsin,55095.0,2588,20.0
816617,2020-12-10,Portage,Wisconsin,55097.0,5325,45.0
816618,2020-12-10,Price,Wisconsin,55099.0,892,4.0
816619,2020-12-10,Racine,Wisconsin,55101.0,16952,220.0
816620,2020-12-10,Richland,Wisconsin,55103.0,986,13.0
816621,2020-12-10,Rock,Wisconsin,55105.0,10910,103.0
816622,2020-12-10,Rusk,Wisconsin,55107.0,1025,11.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [36]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,INCOMPLETE
279,12/04/2020,2610,742,174,24,2,2487,3312,161,18,20,516,37,5,439,34,3,709,49,5,695,38,4,380,22,2,371,20,2,693,46,6,669,49,4,311,20,6,313,20,4,6000
280,12/05/2020,1725,606,150,21,3,2485,3307,166,19,21,289,31,3,434,35,3,463,33,6,699,39,5,194,22,2,358,21,2,559,47,6,679,50,5,220,17,4,315,21,4,6000
281,12/06/2020,1540,613,160,14,6,2468,3278,168,19,22,223,39,2,434,36,3,483,45,2,698,40,5,199,18,1,355,21,2,442,36,7,676,49,5,193,22,2,304,21,4,6000
282,12/07/2020,2232,967,194,14,4,2400,3226,172,19,22,392,37,2,427,36,2,628,44,7,670,41,5,315,26,1,346,21,2,651,62,3,671,50,5,246,25,1,287,23,4,6000
283,12/08/2020,1352,784,122,9,13,2155,2943,163,17,22,188,22,2,371,33,3,426,39,3,609,41,5,211,14,0,308,20,2,409,32,2,612,46,4,118,15,2,254,22,3,6000


In [37]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [38]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [39]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [40]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [41]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [42]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [43]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
279,12/04/2020,516,5,New York,Bronx,36005.0
280,12/05/2020,289,3,New York,Bronx,36005.0
281,12/06/2020,223,2,New York,Bronx,36005.0
282,12/07/2020,392,2,New York,Bronx,36005.0
283,12/08/2020,188,2,New York,Bronx,36005.0


In [44]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [45]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [46]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [47]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
279,12/04/2020,709,5,Kings,New York,36047.0
280,12/05/2020,463,6,Kings,New York,36047.0
281,12/06/2020,483,2,Kings,New York,36047.0
282,12/07/2020,628,7,Kings,New York,36047.0
283,12/08/2020,426,3,Kings,New York,36047.0


In [48]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [49]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [50]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [51]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [52]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,7454,50


In [53]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [54]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [55]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
270,11/25/2020,614,1,New York,Queens,36081.0,2020-11-25
271,11/26/2020,184,0,New York,Queens,36081.0,2020-11-26
272,11/27/2020,630,3,New York,Queens,36081.0,2020-11-27
273,11/28/2020,492,2,New York,Queens,36081.0,2020-11-28
274,11/29/2020,463,4,New York,Queens,36081.0,2020-11-29
275,11/30/2020,689,5,New York,Queens,36081.0,2020-11-30
276,12/01/2020,817,4,New York,Queens,36081.0,2020-12-01
277,12/02/2020,792,4,New York,Queens,36081.0,2020-12-02
278,12/03/2020,740,3,New York,Queens,36081.0,2020-12-03
279,12/04/2020,693,6,New York,Queens,36081.0,2020-12-04


In [56]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
234,10/20/2020,Richmond,New York,36085.0,38,0.0,2020-10-20
235,10/21/2020,Richmond,New York,36085.0,52,1.0,2020-10-21
236,10/22/2020,Richmond,New York,36085.0,76,0.0,2020-10-22
237,10/23/2020,Richmond,New York,36085.0,47,1.0,2020-10-23
238,10/24/2020,Richmond,New York,36085.0,32,0.0,2020-10-24
239,10/25/2020,Richmond,New York,36085.0,33,1.0,2020-10-25
240,10/26/2020,Richmond,New York,36085.0,68,0.0,2020-10-26
241,10/27/2020,Richmond,New York,36085.0,76,0.0,2020-10-27
242,10/28/2020,Richmond,New York,36085.0,89,1.0,2020-10-28
243,10/29/2020,Richmond,New York,36085.0,55,0.0,2020-10-29


In [57]:
print (data_date_dt)

2020-12-11 00:00:00


In [58]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
813417,1001.0,42.0
813418,1003.0,141.0


In [59]:
Kings_death.head(2)

,fips,deaths
0,36047.0,5885


In [60]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
816418,51115.0,1.0
816419,51117.0,37.0
816420,51119.0,11.0
816421,51121.0,26.0
816422,51125.0,3.0
816423,51127.0,4.0
816424,51700.0,56.0
816425,51710.0,99.0
816426,51131.0,31.0
816427,51133.0,9.0


In [61]:
covid_death.shape

(3251, 2)

In [62]:
print (County_covid_death)

{1001.0: 42.0, 1003.0: 141.0, 1005.0: 30.0, 1007.0: 40.0, 1009.0: 47.0, 1011.0: 20.0, 1013.0: 42.0, 1015.0: 129.0, 1017.0: 55.0, 1019.0: 24.0, 1021.0: 47.0, 1023.0: 20.0, 1025.0: 23.0, 1027.0: 27.0, 1029.0: 14.0, 1031.0: 25.0, 1033.0: 53.0, 1035.0: 15.0, 1037.0: 4.0, 1039.0: 38.0, 1041.0: 31.0, 1043.0: 57.0, 1045.0: 59.0, 1047.0: 34.0, 1049.0: 48.0, 1051.0: 72.0, 1053.0: 32.0, 1055.0: 74.0, 1057.0: 16.0, 1059.0: 34.0, 1061.0: 12.0, 1063.0: 18.0, 1065.0: 33.0, 1067.0: 7.0, 1069.0: 46.0, 1071.0: 24.0, 1073.0: 548.0, 1075.0: 8.0, 1077.0: 60.0, 1079.0: 36.0, 1081.0: 72.0, 1083.0: 49.0, 1085.0: 30.0, 1087.0: 23.0, 1089.0: 163.0, 1091.0: 24.0, 1093.0: 39.0, 1095.0: 64.0, 1097.0: 386.0, 1099.0: 11.0, 1101.0: 255.0, 1103.0: 62.0, 1105.0: 9.0, 1107.0: 23.0, 1109.0: 16.0, 1111.0: 27.0, 1113.0: 6.0, 1117.0: 86.0, 1115.0: 60.0, 1119.0: 22.0, 1121.0: 61.0, 1123.0: 92.0, 1125.0: 191.0, 1127.0: 113.0, 1129.0: 22.0, 1131.0: 18.0, 1133.0: 25.0, 2013.0: 0.0, 2016.0: 0.0, 2020.0: 85.0, 2050.0: 7.0, 2060.

In [63]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [64]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [65]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
771540,2020-11-27,New Haven,Connecticut,9009.0,27925,1238.0,2020-11-27,-14 days,NaN
774785,2020-11-28,New Haven,Connecticut,9009.0,27925,1238.0,2020-11-28,-13 days,0.0
778031,2020-11-29,New Haven,Connecticut,9009.0,27925,1238.0,2020-11-29,-12 days,0.0
781277,2020-11-30,New Haven,Connecticut,9009.0,29455,1256.0,2020-11-30,-11 days,1530.0
784522,2020-12-01,New Haven,Connecticut,9009.0,30143,1259.0,2020-12-01,-10 days,688.0
787766,2020-12-02,New Haven,Connecticut,9009.0,30819,1275.0,2020-12-02,-9 days,676.0
791012,2020-12-03,New Haven,Connecticut,9009.0,32377,1283.0,2020-12-03,-8 days,1558.0
794258,2020-12-04,New Haven,Connecticut,9009.0,32843,1291.0,2020-12-04,-7 days,466.0
797503,2020-12-05,New Haven,Connecticut,9009.0,32843,1291.0,2020-12-05,-6 days,0.0
800749,2020-12-06,New Haven,Connecticut,9009.0,32843,1291.0,2020-12-06,-5 days,0.0


In [66]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [67]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,8429.0


In [68]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
234,10/20/2020,Richmond,New York,36085.0,38,0.0,2020-10-20
235,10/21/2020,Richmond,New York,36085.0,52,1.0,2020-10-21
236,10/22/2020,Richmond,New York,36085.0,76,0.0,2020-10-22
237,10/23/2020,Richmond,New York,36085.0,47,1.0,2020-10-23
238,10/24/2020,Richmond,New York,36085.0,32,0.0,2020-10-24
239,10/25/2020,Richmond,New York,36085.0,33,1.0,2020-10-25
240,10/26/2020,Richmond,New York,36085.0,68,0.0,2020-10-26
241,10/27/2020,Richmond,New York,36085.0,76,0.0,2020-10-27
242,10/28/2020,Richmond,New York,36085.0,89,1.0,2020-10-28
243,10/29/2020,Richmond,New York,36085.0,55,0.0,2020-10-29


In [69]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [70]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [71]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [72]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [73]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3217.0,72149.0,40.0,22993,72149,PUERTO RICO
3216,3218.0,72151.0,39.0,34149,72151,PUERTO RICO
3217,3219.0,72153.0,62.0,36439,72153,PUERTO RICO
3218,0.0,0.0,0.0,2132,2105,ALASKA
3219,0.0,0.0,0.0,689,2282,ALASKA


In [74]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2020-12-12 04:02:16.621958
12


In [75]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [76]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [77]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [78]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [79]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 42.0
1003.0 141.0
1005.0 30.0
1007.0 40.0
1009.0 47.0
1011.0 20.0
1013.0 42.0
1015.0 129.0
1017.0 55.0
1019.0 24.0
1021.0 47.0
1023.0 20.0
1025.0 23.0
1027.0 27.0
1029.0 14.0
1031.0 25.0
1033.0 53.0
1035.0 15.0
1037.0 4.0
1039.0 38.0
1041.0 31.0
1043.0 57.0
1045.0 59.0
1047.0 34.0
1049.0 48.0
1051.0 72.0
1053.0 32.0
1055.0 74.0
1057.0 16.0
1059.0 34.0
1061.0 12.0
1063.0 18.0
1065.0 33.0
1067.0 7.0
1069.0 46.0
1071.0 24.0
1073.0 548.0
1075.0 8.0
1077.0 60.0
1079.0 36.0
1081.0 72.0
1083.0 49.0
1085.0 30.0
1087.0 23.0
1089.0 163.0
1091.0 24.0
1093.0 39.0
1095.0 64.0
1097.0 386.0
1099.0 11.0
1101.0 255.0
1103.0 62.0
1105.0 9.0
1107.0 23.0
1109.0 16.0
1111.0 27.0
1113.0 6.0
1117.0 86.0
1115.0 60.0
1119.0 22.0
1121.0 61.0
1123.0 92.0
1125.0 191.0
1127.0 113.0
1129.0 22.0
1131.0 18.0
1133.0 25.0
2013.0 0.0
2016.0 0.0
2020.0 85.0
2050.0 7.0
2060.0 0.0
2068.0 0.0
2070.0 1.0
2090.0 18.0
2100.0 0.0
2110.0 3.0
2122.0 12.0
2130.0 0.0
2150.0 1.0
2158.0 2.0
2164.0 0.0
2170.0 8.0
2180.0 0.0
218

25015.0 170.0
25017.0 2507.0
25019.0 2.0
25021.0 1204.0
25023.0 925.0
25025.0 1291.0
nan 6.0
25027.0 1343.0
26001.0 8.0
26003.0 2.0
26005.0 42.0
26007.0 19.0
26009.0 8.0
26011.0 22.0
26013.0 25.0
26015.0 20.0
26017.0 133.0
26019.0 12.0
26021.0 135.0
26023.0 70.0
26025.0 145.0
26027.0 34.0
26029.0 7.0
26031.0 24.0
26033.0 15.0
26035.0 27.0
26037.0 29.0
26039.0 5.0
26041.0 65.0
26043.0 60.0
26045.0 62.0
26047.0 29.0
26049.0 483.0
26051.0 14.0
26053.0 25.0
26055.0 28.0
26057.0 46.0
26059.0 49.0
26061.0 16.0
26063.0 23.0
26065.0 140.0
26067.0 36.0
26069.0 34.0
26071.0 36.0
26073.0 31.0
26075.0 139.0
26077.0 190.0
26079.0 9.0
26081.0 441.0
26083.0 2.0
26085.0 6.0
26087.0 75.0
26089.0 3.0
26091.0 49.0
26093.0 57.0
26095.0 0.0
26097.0 1.0
26099.0 1394.0
26101.0 10.0
26103.0 41.0
26105.0 13.0
26107.0 11.0
26109.0 21.0
26111.0 31.0
26113.0 7.0
26115.0 114.0
26117.0 39.0
26119.0 3.0
26121.0 217.0
26123.0 24.0
26125.0 1462.0
26127.0 30.0
26129.0 22.0
26131.0 16.0
26133.0 13.0
26135.0 11.0
26137.0

48273.0 55.0
48275.0 10.0
48283.0 14.0
48277.0 75.0
48279.0 55.0
48281.0 11.0
48285.0 44.0
48287.0 21.0
48289.0 18.0
48291.0 72.0
48293.0 20.0
48295.0 5.0
48297.0 10.0
48299.0 5.0
48301.0 0.0
48303.0 436.0
48305.0 11.0
48313.0 15.0
48315.0 17.0
48317.0 9.0
48319.0 2.0
48321.0 63.0
48323.0 179.0
48307.0 8.0
48309.0 229.0
48311.0 1.0
48325.0 51.0
48327.0 3.0
48329.0 155.0
48331.0 10.0
48333.0 8.0
48335.0 7.0
48337.0 27.0
48339.0 222.0
48341.0 39.0
48343.0 8.0
48345.0 2.0
48347.0 89.0
48349.0 46.0
48351.0 16.0
48353.0 12.0
48355.0 507.0
48357.0 9.0
48359.0 2.0
48361.0 58.0
48363.0 40.0
48365.0 36.0
48367.0 74.0
48369.0 28.0
48371.0 20.0
48373.0 43.0
48375.0 257.0
48377.0 11.0
48379.0 10.0
48381.0 153.0
48383.0 6.0
48385.0 10.0
48387.0 18.0
48389.0 17.0
48391.0 16.0
48393.0 1.0
48395.0 9.0
48397.0 42.0
48399.0 12.0
48401.0 38.0
48403.0 11.0
48405.0 19.0
48407.0 16.0
48409.0 87.0
48411.0 11.0
48413.0 5.0
48415.0 26.0
48417.0 0.0
48419.0 37.0
48421.0 3.0
48423.0 215.0
48425.0 5.0
48427.0 192

In [80]:
K_c.head()

,fips,cases,deaths
0,36047.0,7454,50


In [81]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [82]:
print (K)

{36047.0: 7454}


In [83]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 430.0
1001 470.0
1009 712.0
1013 134.0
1015 1294.0
1017 212.0
1031 395.0
1033 593.0
1039 280.0
1043 1120.0
1045 357.0
1051 758.0
1055 1483.0
1067 104.0
1069 677.0
1071 1054.0
1077 913.0
1079 252.0
1083 862.0
1089 3050.0
1095 1338.0
1097 2017.0
1103 1591.0
1111 96.0
1113 216.0
1115 900.0
1117 2051.0
1121 690.0
2261 40.0
4021 3831.0
5009 343.0
5011 162.0
5033 536.0
5037 198.0
5045 1002.0
5047 127.0
5051 668.0
5053 126.0
5063 258.0
5083 180.0
5085 476.0
5087 176.0
5115 536.0
5117 40.0
5121 208.0
5125 1127.0
5131 915.0
5145 427.0
6007 935.0
6017 1232.0
6023 345.0
6027 67.0
6061 2506.0
6079 1138.0
6089 1657.0
6093 276.0
8019 40.0
8039 205.0
8047 23.0
8051 57.0
8057 1.0
8065 79.0
8067 530.0
8071 146.0
8075 488.0
8077 1612.0
8083 244.0
8085 471.0
8093 62.0
8099 251.0
8101 3086.0
8103 67.0
8119 185.0
8121 93.0
9005 1479.0
9007 1053.0
9011 1618.0
9015 898.0
10001 1738.0
12019 1053.0
12033 1768.0
12089 485.0
12101 2500.0
12109 1403.0
12113 1126.0
13015 858.0
13039 208.0
13045 830.0
13047 4

38099 110.0
39003 1400.0
39005 665.0
39007 676.0
39011 542.0
39013 633.0
39015 543.0
39017 4538.0
39019 223.0
39021 358.0
39023 1513.0
39025 2235.0
39027 530.0
39029 1072.0
39033 532.0
39037 853.0
39039 507.0
39043 1004.0
39045 1673.0
39051 569.0
39053 379.0
39055 820.0
39057 1692.0
39059 458.0
39063 856.0
39065 320.0
39067 161.0
39069 307.0
39071 474.0
39073 308.0
39077 838.0
39081 920.0
39083 540.0
39085 2209.0
39089 2106.0
39091 507.0
39093 3535.0
39099 3111.0
39103 1907.0
39107 452.0
39109 1353.0
39111 150.0
39113 6150.0
39117 357.0
39119 1190.0
39123 391.0
39125 225.0
39127 454.0
39133 1541.0
39135 524.0
39137 460.0
39139 1861.0
39141 720.0
39143 810.0
39147 627.0
39149 594.0
39151 4884.0
39153 5878.0
39155 2389.0
39157 1160.0
39159 658.0
39161 317.0
39165 2918.0
39167 741.0
39169 1173.0
39171 499.0
39175 308.0
40013 371.0
40015 499.0
40017 1497.0
40019 341.0
40021 463.0
40027 2476.0
40031 937.0
40033 101.0
40035 205.0
40037 485.0
40045 79.0
40047 666.0
40049 238.0
40051 548.0
400

27113 199.0
27117 122.0
27119 582.0
27131 774.0
27139 1824.0
27141 1282.0
27145 2366.0
27149 116.0
27155 30.0
27159 222.0
27165 158.0
27169 556.0
27171 2326.0
28019 81.0
28037 90.0
28067 613.0
28073 430.0
28075 635.0
28077 96.0
28085 258.0
28131 164.0
28137 311.0
28139 237.0
28149 272.0
28155 102.0
29005 38.0
29009 159.0
29017 96.0
29019 1413.0
29027 448.0
29035 42.0
29053 137.0
29065 96.0
29067 60.0
29077 2494.0
29093 42.0
29095 2972.0
29101 332.0
29111 46.0
29117 82.0
29123 151.0
29135 134.0
29147 214.0
29163 193.0
29165 315.0
29169 472.0
29175 192.0
29179 56.0
29187 547.0
29201 417.0
29207 235.0
29209 192.0
29211 30.0
29213 481.0
29225 226.0
30011 20.0
30019 23.0
30037 1.0
30039 19.0
30055 26.0
30057 50.0
30065 38.0
30077 77.0
30079 39.0
30087 101.0
30089 91.0
30093 502.0
30097 40.0
30109 8.0
31009 0.0
31013 86.0
31015 12.0
31017 26.0
31029 27.0
31045 41.0
31049 19.0
31055 6879.0
31057 22.0
31061 12.0
31063 17.0
31069 10.0
31071 9.0
31073 22.0
31077 37.0
31083 24.0
31087 43.0
31097 

21109 70.0
21121 235.0
21129 85.0
21203 170.0
24510 3243.0
26085 61.0
28013 145.0
29223 80.0
30035 42.0
31043 266.0
31047 260.0
34011 910.0
34017 5303.0
34031 5071.0
34039 3849.0
35005 1221.0
35019 131.0
35023 24.0
35033 27.0
35045 1637.0
35051 111.0
35053 153.0
36047 600.0
36081 563.0
36087 2247.0
37083 279.0
38005 109.0
40001 215.0
40139 266.0
41049 77.0
45009 63.0
45033 202.0
45053 68.0
45075 407.0
45089 148.0
47007 146.0
47061 129.0
48003 137.0
48013 262.0
48017 42.0
48025 118.0
48069 55.0
48079 13.0
48095 7.0
48101 14.0
48103 16.0
48113 20565.0
48117 502.0
48153 28.0
48169 12.0
48177 105.0
48195 27.0
48201 14994.0
48207 11.0
48249 171.0
48255 42.0
48271 13.0
48279 200.0
48301 0.0
48341 126.0
48357 157.0
48369 90.0
48371 107.0
48375 1708.0
48389 106.0
48419 20.0
48435 22.0
48437 45.0
48463 160.0
48465 376.0
48501 72.0
51027 150.0
51051 195.0
51105 181.0
51183 12.0
51195 253.0
51640 97.0
51710 771.0
51720 27.0
54005 142.0
54015 77.0
54043 104.0
1047 208.0
1087 92.0
1119 60.0
2016 43

In [84]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 3236
36061 3882
36047 7454 7454
36081 7377
36005 4528


In [85]:
print (covid_cases_county_ny_times)

{1001.0: 470.0, 1003.0: 1685.0, 1005.0: 87.0, 1007.0: 186.0, 1009.0: 712.0, 1011.0: 29.0, 1013.0: 134.0, 1015.0: 1294.0, 1017.0: 212.0, 1019.0: 134.0, 1021.0: 288.0, 1023.0: 23.0, 1025.0: 175.0, 1027.0: 80.0, 1029.0: 87.0, 1031.0: 395.0, 1033.0: 593.0, 1035.0: 86.0, 1037.0: 82.0, 1039.0: 280.0, 1041.0: 50.0, 1043.0: 1120.0, 1045.0: 357.0, 1047.0: 208.0, 1049.0: 822.0, 1051.0: 758.0, 1053.0: 235.0, 1055.0: 1483.0, 1057.0: 219.0, 1059.0: 298.0, 1061.0: 161.0, 1063.0: 94.0, 1065.0: 138.0, 1067.0: 104.0, 1069.0: 677.0, 1071.0: 1054.0, 1073.0: 6850.0, 1075.0: 135.0, 1077.0: 913.0, 1079.0: 252.0, 1081.0: 764.0, 1083.0: 862.0, 1085.0: 55.0, 1087.0: 92.0, 1089.0: 3050.0, 1091.0: 164.0, 1093.0: 253.0, 1095.0: 1338.0, 1097.0: 2017.0, 1099.0: 108.0, 1101.0: 1159.0, 1103.0: 1591.0, 1105.0: 45.0, 1107.0: 147.0, 1109.0: 129.0, 1111.0: 96.0, 1113.0: 216.0, 1115.0: 900.0, 1117.0: 2051.0, 1119.0: 60.0, 1121.0: 690.0, 1123.0: 261.0, 1125.0: 1948.0, 1127.0: 659.0, 1129.0: 83.0, 1131.0: 62.0, 1133.0: 290.

In [86]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [87]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [88]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [89]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1)  
    return state_dict



In [90]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [91]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [92]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [93]:
#ACI
ACI_State = total_state(ACI_total)

In [94]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [95]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [96]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [97]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 957116
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 247333
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1780968
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 248591
PUERTO RICO 1622194


In [98]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

In [99]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [100]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 27.0 38921 69.37129056293517
1001 42.0 55869 75.17585781023465
1009 47.0 57826 81.27831771175595
1013 42.0 19448 215.9605100781571
1015 129.0 113605 113.5513401698869
1017 55.0 33254 165.3936368557166
1031 25.0 52342 47.76279087539643
1033 53.0 55241 95.9432305714958
1039 38.0 37049 102.56687090069907
1043 57.0 83768 68.04507687899914
1045 59.0 49172 119.98698446270235
1051 72.0 81209 88.66012387789532
1055 74.0 102268 72.3589001447178
1067 7.0 17205 40.68584713746004
1069 46.0 105882 43.444589259741974
1071 24.0 51626 46.48820361833185
1077 60.0 92729 64.70467706974085
1079 36.0 32924 109.34272870854088
1083 49.0 98915 49.53748167618663
1089 163.0 372909 43.71039583383614
1095 64.0 96774 66.13346560026453
1097 386.0 413210 93.41497059606495
1103 62.0 119679 51.80524569891125
1111 27.0 22722 118.82756799577501
1113 6.0 57961 10.351788271423889
1115 60.0 89512 67.03011886674413
1117 86.0 217702 39.50354153843327
1121 61.0 79978 76.27097451799244
2261 3.0 9202 32.60160834601174
402

29173 2.0 10309 19.400523814142982
29177 6.0 23018 26.066556607872098
29183 244.0 402022 60.69319589475202
29186 2.0 17894 11.176930814798256
29189 1143.0 994205 114.96622929878647
29195 14.0 22761 61.508721057950005
29205 1.0 5930 16.863406408094434
29217 13.0 20563 63.220347225599376
29219 14.0 35649 39.271788829981205
29227 0.0 2013 0.0
30001 3.0 9453 31.735956839098698
30007 1.0 6237 16.0333493666827
30009 7.0 10725 65.26806526806527
30013 80.0 81366 98.32116608902982
30015 3.0 5635 53.23868677905945
30017 8.0 11402 70.1631292755657
30021 16.0 8613 185.76570300708232
30025 1.0 2846 35.137034434293746
30027 10.0 11050 90.49773755656109
30029 44.0 103806 42.386759917538484
30031 30.0 114434 26.21598475977419
30033 2.0 1258 158.9825119236884
30043 1.0 12221 8.182636445462729
30045 0.0 2007 0.0
30047 15.0 30458 49.24814498653885
30049 24.0 69432 34.566194262011756
30059 7.0 1862 375.9398496240601
30063 45.0 119600 37.625418060200666
30067 4.0 16606 24.087679152113694
30069 1.0 487 205.

42117 35.0 40591 86.22601069202533
42121 16.0 50668 31.57811636535881
42123 2.0 39191 5.103212472251283
42125 85.0 206865 41.089599497256664
42127 13.0 51361 25.311033663674774
42129 273.0 348899 78.24614000040125
42131 29.0 26794 108.23318653429874
42133 280.0 449058 62.352747306584
44001 49.0 48479 101.07469213473875
44005 9.0 82082 10.964645110986574
44009 95.0 125577 75.65079592600556
45003 109.0 170872 63.79043962732337
45007 241.0 202558 118.97826795288263
45015 106.0 227907 46.51019933569395
45023 38.0 32244 117.85138320307654
45035 115.0 162809 70.63491576018525
45045 428.0 523542 81.75084329432977
45055 70.0 66551 105.18249162296584
45057 70.0 98012 71.4198261437375
45063 242.0 298750 81.00418410041841
45073 45.0 79546 56.57104065572121
45077 100.0 126884 78.81214337505124
45083 306.0 319785 95.6892912425536
45091 149.0 280979 53.0288740439677
46015 5.0 5297 94.39305267132339
46019 15.0 10429 143.8297056285358
46025 1.0 3736 26.76659528907923
46029 57.0 28009 203.5060159234532

13103 34.0 64296 52.880428020405624
13105 30.0 19194 156.29884338855894
13117 66.0 244252 27.021273111376775
13131 31.0 24633 125.84744042544553
13133 28.0 18324 152.80506439642
13137 84.0 45328 185.31591951994352
13149 8.0 11923 67.09720707875535
13159 7.0 14219 49.22990365004571
13171 29.0 19077 152.01551606646748
13177 33.0 29992 110.02934115764204
13181 11.0 7921 138.87135462694104
13185 130.0 117406 110.7268793758411
13195 21.0 29880 70.28112449799197
13201 3.0 5718 52.46589716684156
13219 37.0 40280 91.85700099304866
13221 14.0 15259 91.74913166000394
13223 89.0 168667 52.766694136968106
13237 31.0 22119 140.15100140151003
13241 12.0 17137 70.02392484098733
13255 88.0 66703 131.92809918594367
13257 48.0 25925 185.14946962391514
13275 83.0 44451 186.72245843738048
13281 33.0 12037 274.1546897067376
13285 126.0 69922 180.20079517176282
13291 40.0 24511 163.19203622863202
13293 80.0 26320 303.951367781155
13305 49.0 29927 163.73174725164566
13317 10.0 9777 102.28086325048584
16005 4

29077 247.0 293086 84.27560511249257
29093 1.0 10125 9.876543209876543
29095 196.0 703011 27.88007584518592
29101 21.0 54062 38.84428988938626
29111 2.0 9776 20.458265139116204
29117 17.0 15227 111.64379063505615
29123 10.0 12088 82.7266710787558
29135 15.0 16132 92.98289114802877
29147 17.0 22092 76.95093246424045
29163 11.0 18302 60.102721014096815
29165 17.0 104418 16.28071788388975
29169 24.0 52607 45.62130514950482
29175 10.0 24748 40.40730564085987
29179 2.0 6270 31.89792663476874
29187 51.0 67215 75.87592055344788
29201 44.0 38280 114.94252873563218
29207 25.0 29025 86.13264427217915
29209 20.0 31952 62.59389083625439
29211 8.0 6089 131.3844637871572
29213 46.0 55928 82.24860534973537
29225 29.0 39592 73.24712063043039
30011 4.0 1252 319.4888178913738
30019 5.0 1690 295.85798816568047
30037 0.0 821 0.0
30039 1.0 3379 29.59455460195324
30055 0.0 1664 0.0
30057 3.0 8600 34.883720930232556
30065 6.0 4633 129.5057198359594
30077 4.0 6890 58.05515239477503
30079 2.0 1077 185.70102135

6005 18.0 39752 45.28074059166835
6015 1.0 27812 3.5955702574428305
6051 3.0 14444 20.76986984214899
6055 23.0 137744 16.697642002555465
6087 50.0 273213 18.30073971589932
6097 162.0 494336 32.77123252200932
6103 30.0 65084 46.09427816360396
6111 197.0 846006 23.285886861322496
8033 0.0 2055 0.0
8037 12.0 55127 21.76791771727103
8045 14.0 60061 23.309635204209055
8053 0.0 820 0.0
8055 11.0 6897 159.4896331738437
8073 1.0 5701 17.540782318891424
8087 62.0 29068 213.29296821246731
8109 4.0 6824 58.61664712778428
8111 0.0 728 0.0
8117 5.0 31011 16.123311083164037
8125 3.0 10019 29.94310809462022
9001 1581.0 943332 167.59741003167494
9009 1351.0 854757 158.05661726081215
12003 34.0 29210 116.39849366655255
12007 22.0 28201 78.01141803482146
12023 103.0 71686 143.68216946126162
12055 180.0 106221 169.45801677634367
12057 977.0 1471968 66.3737255157721
12061 143.0 159923 89.41803242810603
12065 12.0 14246 84.23417099536712
12081 393.0 403253 97.45742747109135
12085 185.0 161000 114.906832298

51013 164.0 236842 69.24447521976676
51021 5.0 6280 79.61783439490446
51025 8.0 16231 49.28839874314583
51059 632.0 1147532 55.07471687064064
51107 151.0 413538 36.51417765719233
51111 3.0 12196 24.59822892751722
51135 24.0 15232 157.56302521008405
51141 28.0 17608 159.01862789641072
51167 10.0 26586 37.61378168961107
51175 40.0 17631 226.873121206965
51510 82.0 159428 51.433876107082824
51530 1.0 6478 15.436863229391788
51600 10.0 24019 41.63370664890295
51630 6.0 29036 20.664003306240527
51650 38.0 134510 28.25068768121329
51775 25.0 25301 98.81032370262047
51830 8.0 14954 53.49739200213989
51840 5.0 28078 17.807536149298382
53013 3.0 3985 75.2823086574655
53017 11.0 43429 25.328697414170254
53019 1.0 7627 13.111315064901008
53047 31.0 42243 73.38493951660631
53049 3.0 22471 13.350540696898223
53051 3.0 13724 21.85951617604197
53057 31.0 129205 23.99287953252583
53069 0.0 4488 0.0
54055 45.0 58758 76.58531604207087
54075 2.0 8247 24.251242876197406
54089 11.0 12573 87.48906386701663


54005 17.0 21457 79.22822388963974
54015 3.0 8508 35.26093088857546
54043 6.0 20409 29.398794649419372
1047 34.0 37196 91.40767824497257
1087 23.0 18068 127.2968784591543
1119 22.0 12427 177.03387784662428
2016 0.0 5634 0.0
2070 1.0 4916 20.34174125305126
2164 0.0 1592 0.0
2180 0.0 10004 0.0
4017 286.0 110924 257.8341927806426
4023 81.0 46498 174.20104090498515
4027 417.0 213787 195.05395557260263
5017 30.0 10118 296.501284838901
5123 17.0 24994 68.01632391774027
6011 7.0 21547 32.48712117696199
8023 2.0 3887 51.45356315924878
12043 11.0 13811 79.6466584606473
12067 20.0 8422 237.47328425552126
13043 35.0 10803 323.9840784967139
13061 3.0 2834 105.85744530698659
13065 13.0 6618 196.43396796615292
13161 34.0 15115 224.94211048627193
13163 40.0 15362 260.3827626611118
13205 52.0 21863 237.84476055436124
13243 31.0 6778 457.36205370315724
13271 31.0 15860 195.46027742749055
13273 34.0 8531 398.5464775524557
20055 32.0 36467 87.7505690075959
20175 13.0 21428 60.66828448758634
21063 1.0 751

In [101]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(1351.0, 854757, 158.05661726081215)

In [102]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [103]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [104]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 957116.0000000001, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 232454.99999999994, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 247333.00000000003, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [105]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [106]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76256043.0 1000000 10125.12542776446
ALABAMA 957116.0000000001 76256043.0 1000000 12551.3462585516
ALASKA 231145.0 76256043.0 1000000 3031.1696084204104
ARIZONA 1839932.0 76256043.0 1000000 24128.343507149984
ARKANSAS 830467.0000000003 76256043.0 1000000 10890.507392312506
CALIFORNIA 11847711.0 76256043.0 1000000 155367.5031367678
COLORADO 1337805.0 76256043.0 1000000 17543.593233653628
CONNECTICUT 874974.0 76256043.0 1000000 11474.159497103725
DELAWARE 239009.0 76256043.0 1000000 3134.295861640762
FLORIDA 3892551.999999999 76256043.0 1000000 51045.81678857896
GEORGIA 1928702.999999999 76256043.0 1000000 25292.461084034992
IDAHO 340742.0000000001 76256043.0 1000000 4468.393409818027
ILLINOIS 2987496.0000000005 76256043.0 1000000 39177.170522734836
INDIANA 1602975.9999999995 76256043.0 1000000 21020.97010200227
IOWA 699741.0 76256043.0 1000000 9176.203910816615
KANSAS 401103.0000000001 76256043.0 1000000 5259.950349115284
KENTUCKY 1416013.0000000002 76256043.0 100000

In [107]:
print (Covid_state)

{'NEW MEXICO': 22881.0, 'ALABAMA': 42050.0, 'ALASKA': 8477.0, 'ARIZONA': 71352.0, 'ARKANSAS': 25018.0, 'CALIFORNIA': 298036.0, 'COLORADO': 56394.0, 'CONNECTICUT': 30006.0, 'DELAWARE': 8882.0, 'FLORIDA': 117048.0, 'GEORGIA': 58459.0, 'IDAHO': 19213.0, 'ILLINOIS': 119054.0, 'INDIANA': 80529.0, 'IOWA': 25899.0, 'KANSAS': 28510.0, 'KENTUCKY': 42453.0, 'LOUISIANA': 30821.0, 'MAINE': 3606.0, 'MARYLAND': 32968.0, 'MASSACHUSETTS': 52537.0, 'MICHIGAN': 87500.0, 'MINNESOTA': 72203.0, 'MISSISSIPPI': 24569.0, 'MISSOURI': 45336.0, 'MONTANA': 11175.0, 'NEBRASKA': 21775.0, 'NEVADA': 32345.0, 'NEW HAMPSHIRE': 8829.0, 'NEW JERSEY': 59974.0, 'NEW YORK': 102080.0, 'NORTH CAROLINA': 62300.0, 'NORTH DAKOTA': 9483.0, 'OHIO': 132042.0, 'OKLAHOMA': 37928.0, 'OREGON': 19014.0, 'PENNSYLVANIA': 113817.0, 'RHODE ISLAND': 14087.0, 'SOUTH CAROLINA': 28566.0, 'SOUTH DAKOTA': 10456.0, 'TENNESSEE': 70285.0, 'TEXAS': 175475.0, 'UTAH': 36042.0, 'VERMONT': 1408.0, 'VIRGINIA': 40776.0, 'WASHINGTON': 36015.0, 'WEST VIRGINI

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [108]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [109]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [110]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [111]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [112]:
#print (Medicaid_demand)

In [113]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [114]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)#{m: Unemployment_capita[m] for m in location}

In [115]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [116]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)